In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import data

In [59]:
verbose = False
path = './data/2013.txt'
corpus = data.Corpus(path, verbose)

In [218]:
class Model(nn.Module):
        
    def __init__(self, vocab_size, embed_size, nhidden, nlayers):
        super(Model, self).__init__()
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, nhidden, nlayers)
        self.decoder = nn.Linear(nhidden, vocab_size)

    def forward(self, x, h0):
        y = self.encoder(x)
        y = y.unsqueeze(0)
        y, h1 = self.rnn(y, h0)
        y = self.decoder(y)
        return y, h1

In [216]:
vocab_size = len(corpus)
embed_size = 200
nhidden = 10
nlayers = 2
lm = Model(vocab_size, embed_size, nhidden, nlayers)
lm

Model(
  (encoder): Embedding(27738, 200)
  (rnn): RNN(200, 10, num_layers=2)
  (decoder): Linear(in_features=10, out_features=27738, bias=True)
)

In [217]:
# for value in corpus.data:
x = torch.tensor([0,1,2,3])
h0 = torch.zeros((nhidden, 4, 10))
lm(x, h0)

torch.Size([4])
torch.Size([4, 200])
torch.Size([1, 4, 200])
torch.Size([1, 4, 10])


tensor([[[-0.0497,  0.2228, -0.1026,  ..., -0.0213, -0.1622,  0.0666],
         [-0.3541,  0.1122, -0.4661,  ...,  0.2050, -0.3343, -0.0287],
         [ 0.2209,  0.2727, -0.4521,  ...,  0.1353, -0.0984, -0.1304],
         [-0.3459, -0.0217, -0.3512,  ...,  0.4322, -0.4496, -0.1487]]],
       grad_fn=<AddBackward0>)

In [202]:
torch.zeros((1,2,3))

tensor([[[0., 0., 0.],
         [0., 0., 0.]]])